In [1]:
import os
import torch
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
import os
import torch
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_bounds
from shapely.geometry import box
import natcap.invest.habitat_quality as habitat_quality
import natcap.invest.urban_cooling_model as urban_cooling_model
import natcap.invest.urban_nature_access as urban_nature_access

d:\Anaconda\envs\INV\lib\site-packages\natcap\invest\__init__.py:34: FutureWarning: 
        natcap.invest requires GDAL exceptions to be enabled. You must
        call gdal.UseExceptions() to avoid unexpected behavior from
        natcap.invest. A future version will enable exceptions on import.
        gdal.UseExceptions() affects global state, so this may affect the
        behavior of other packages.
  warnings.warn(('''


# Run InVEST model 
- This code creates InVEST model inputs from processed pytorch files, run InVEST model for calculation outputs, then pack outputs for later deep learning training.

In [2]:
os.chdir("E:/Coding/CNN_InV_ReOrg/")
lulc = 'InVEST_Model/MainInputs/lulc.tif'
work_dir_hq = 'InVEST_Model/HabitatQuality/WorkDir'
threats_p = 'InVEST_Model/HabitatQuality/threat.csv'
sensitivity_p = 'InVEST_Model/HabitatQuality/sensitivity.csv'
work_dir_ucm = 'InVEST_Model/UrbanCooling/WorkDir'
aoi = 'InVEST_Model/MainInputs/aoi.shp'
et0 = 'InVEST_Model/MainInputs/et0.tif'
pop = 'InVEST_Model/MainInputs/pop.tif'
bio_csv = 'InVEST_Model/UrbanCooling/Biophysical_ucm_real.csv'
work_dir_una = 'InVEST_Model/UrbanNatureAccess/WorkDir'
attr_csv = "InVEST_Model/UrbanNatureAccess/attr_unacsv.csv"
# This is the path of the pre-processed tensor files
input_dir = 'Prosd'

# This is the path of the temporary InVEST model inputs
output_dir = "InVEST_Model/MainInputs"
thrt_output_dir = "InVEST_Model/HabitatQuality"
final_output_dir = "Results_updt"
error_pop = "Error_pop"
os.makedirs(error_pop, exist_ok=True)

In [3]:
def load_and_plot_pt_data(file_path):
    """
    Load a .pt file, reshape the data to (3, 512, 512), and plot each channel in 3 subplots.

    Parameters:
    - file_path: str, path to the .pt file.
    """
    # Load the .pt file with weights_only=True for safety
    data = torch.load(file_path, weights_only=True)

    # Ensure the data has the expected shape
    if data.shape != (3, 1, 512, 512):
        raise ValueError(f"Expected data shape (3, 1, 512, 512), but got {data.shape}")

    # Squeeze the second dimension to reshape the data to (3, 512, 512)
    data = data.squeeze(1)  # Removes the dimension of size 1

    # Plot each channel
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    for i in range(3):
        axs[i].imshow(data[i].numpy(), cmap='viridis')
        axs[i].set_title(f'Channel {i+1}')
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()


## InVEST Input processing

In [4]:
# Load sample grids as a GeoDataFrame
samples = gpd.read_file('Sample/Samplefishnets_32618.shp')

def get_expanded_grid(cell, expand_size=1280):
    # Find the center of the cell
    minx, miny, maxx, maxy = cell.bounds
    center_x = (minx + maxx) / 2
    center_y = (miny + maxy) / 2

    # Define the expanded box around the center
    expanded_bounds = box(
        center_x - expand_size,
        center_y - expand_size,
        center_x + expand_size,
        center_y + expand_size
    )

    return expanded_bounds

# This function is used to process the saved tensor data to GeoTIFF and shapefile format for InVEST inputs
# Currently, it only supports the specified models: habitat_quality, urban_cooling_model, and urban_nature_access
def process_samples(samples, torc, fid, output_dir, thrt_output_dir, expand_size=1280, threats=[(7, 10), (8, 11), (9, 12)], channel_names = ['lulc', 'et0', 'pop']):
    """
    Load samples, match tensor by FID, expand bounds, and save as GeoTIFF and shapefile.

    Parameters:
    - samples: GeoDataFrame, loaded sample shapefile
    - torc: torch.Tensor, indicating data to convert
    - fid: int, feature ID to match in the sample shapefile
    - output_dir: str, directory to save the output files
    - thrt_output_dir: str, directory to save the threat output files
    - expand_size: int, expansion size for the grid cell
    - threats: list of int or tuple, threat values to extract from LULC. Tuple (7, 10) means lulc code 7 and 10 should be considered as one type of threat.
               Threat output name needs to match the threat table. 
    - channel_names: list of str, names of the channels in the tensor data
    Returns:
    - None (saves GeoTIFF files and shapefile to the specified directory)
    """
    # Filter the row in samples that matches the `FID`
    sample_row = samples[samples['id'] == fid]

    if len(sample_row) == 0:
        raise ValueError(f"No matching sample found for FID {fid}.")

    # Get the geometry and CRS
    geometry = sample_row.geometry.values[0]
    crs = samples.crs

    # Expand the bounds
    expanded_grid = get_expanded_grid(geometry, expand_size=expand_size)
    minx, miny, maxx, maxy = expanded_grid.bounds

    # Get the corresponding tensor data
    tensor_data = torc.numpy()

    # Check if tensor_data has 3 channels
    if tensor_data.shape[0] != 3:
        raise ValueError("Tensor data must have 3 channels.")

    # Define transform for the raster (GeoTIFF)
    transform = from_bounds(minx, miny, maxx, maxy, tensor_data.shape[1], tensor_data.shape[2])

    # Save each channel as a separate GeoTIFF

    lulc_index = channel_names.index('lulc')
    lulc_data = tensor_data[lulc_index]  # Extract lulc channel data
    for i, channel_name in enumerate(channel_names):
        output_path = f"{output_dir}/{channel_name}.tif"
            # Check if the channel name is 'lulc'
        if channel_name == 'lulc':
            nodata_value = 255
        else:
            nodata_value = None  # No nodata for other channels

        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=tensor_data.shape[1],
            width=tensor_data.shape[2],
            count=1,  # Single channel per file
            dtype='int32',
            crs=crs,
            transform=transform,
            nodata=nodata_value  # Set nodata value conditionally
        ) as dst:
            dst.write(tensor_data[i].astype(np.int32), 1)

        # print(f"Saved {output_path}")

    # Save the expanded grid as a shapefile
    expanded_grid_gdf = gpd.GeoDataFrame({'id': [fid], 'geometry': [expanded_grid]}, crs=crs)
    output_shapefile_path = f"{output_dir}/aoi.shp"
    expanded_grid_gdf.to_file(output_shapefile_path)

    # print(f"Saved {output_shapefile_path}")

    # Handle threats if provided
    lulc_meta = {
        'driver': 'GTiff',
        'height': lulc_data.shape[0],
        'width': lulc_data.shape[1],
        'count': 1,
        'dtype': 'uint8',
        'crs': crs,
        'transform': transform,
        'compress': 'lzw',
    }
    for threat in threats:
        if isinstance(threat, int):
            # Create binary mask for a single threat value
            binary_mask = np.where(lulc_data == threat, 1, 0).astype(np.uint8)
            out_filename = f"threat_{threat}.tif"
        elif isinstance(threat, tuple):
            # Create binary mask for multiple threat values
            binary_mask = np.isin(lulc_data, threat).astype(np.uint8)
            out_filename = f"threat_{''.join(map(str, threat))}.tif"
        else:
            raise ValueError("Threats must be a list of integers or tuples of integers.")

        # Write the binary mask to a new raster file
        out_path = f"{thrt_output_dir}/{out_filename}"
        with rasterio.open(out_path, 'w', **lulc_meta) as dst:
            dst.write(binary_mask, 1)

        # print(f"Saved binary raster: {out_path}")


In [5]:
# Select one sample to test the function
fid = 1008
test = f"Prosd/{fid}.pt"
torc = torch.load(test, weights_only=True).squeeze(1)
output_dir = "InVEST_Model/MainInputs"
thrt_output_dir = "InVEST_Model/HabitatQuality"
process_samples(samples, torc, fid, output_dir, thrt_output_dir)

In [8]:
# InVEST functions to run the models
def hq(lulc, work_dir, threats_p, sensitivity_p):
    '''
    lulc: string, path to land use land cover raster
    threats: list of ints, each int is a threat, this should contain transportation network
    threat_dir: string, path to directory with threat rasters, including road networks
    output_dir: string, path to output directory
    file_prefix: string, prefix for output files
    '''
    suffix = 'test'
    # run the habitat quality model
    habitat_quality.execute({ 'lulc_cur_path': lulc,
                 'threats_table_path': threats_p,
                 'half_saturation_constant': 0.345,
                 'results_suffix': suffix,
                 'workspace_dir': work_dir,
                 'sensitivity_table_path': sensitivity_p})
    out_path = os.path.join(work_dir, f'quality_c_{suffix}.tif')
    with rxr.open_rasterio(out_path) as hq:
        hq_res = hq.copy()
    return hq_res


def ucm(lulc_path, work_dir_ucm, aoi_p, ref_eto_p, bio_p):
    """
    Run the Urban Cooling Model from InVEST.

    Parameters:
    - lulc_path: str, path to the LULC raster input.
    - work_dir: str, directory to save the model outputs.
    Returns:
    - float, mean value of the output map.
    """
    suffix = 'test'
    args = {
        'workspace_dir': work_dir_ucm,
        'results_suffix': suffix,
        'lulc_raster_path': lulc_path,
        't_ref': 22.6,
        'ref_eto_raster_path': ref_eto_p,
        'aoi_vector_path': aoi_p,
        'biophysical_table_path': bio_p,
        'green_area_cooling_distance': 450.0,
        't_air_average_radius': 200.0,
        'uhi_max': 2.06,
        'do_energy_valuation': False,
        'do_productivity_valuation': False,
        'cc_method': 'factors',
        'cc_weight_shade': 0.6,
        'cc_weight_albedo': 0.2,
        'cc_weight_eti': 0.2,
    }

    urban_cooling_model.execute(args)
    # f"{work_dir_ucm}/hm_{suffix}.tif"
    out_path = os.path.join(work_dir_ucm, f'hm_{suffix}.tif')
    with rxr.open_rasterio(out_path) as ucm:
        ucm_res = ucm.copy()
    return ucm_res


def una(lulc_path, work_dir_una, pop_p, aoi_p, attr_csv):
    """
    Run the Urban Nature Access Model from InVEST.

    Parameters:
    - lulc_path: str, path to the LULC raster input.
    - output_path: str, directory to save the model outputs.
    - population_raster_path: str, path to the population raster input.
    - admin_boundaries_vector_path: str, path to the administrative boundaries vector input.
    - lulc_attribute_table: str, path to the LULC attribute table CSV.
    Returns:
    - float, mean value of the output map.
    """
    suffix = 'test'
    args = {
        'workspace_dir': work_dir_una,
        'results_suffix': suffix,
        'lulc_raster_path': lulc_path,
        'admin_boundaries_vector_path': aoi_p,
        'lulc_attribute_table': attr_csv,
        'population_raster_path': pop_p,
        'urban_nature_demand': 45.0,  # Example value
        'decay_function': 'dichotomy',
        'search_radius_mode': 'radius per urban nature class',
        'aggregate_by_pop_group': False,
    }
    urban_nature_access.execute(args)
    out_path = f"{work_dir_una}/output/urban_nature_balance_totalpop_{suffix}.tif"
    with rxr.open_rasterio(out_path) as una:
        una_res = una.copy()

    return una_res


In [9]:
hq_test = hq(lulc, work_dir_hq, threats_p, sensitivity_p)
ucm_test = ucm(lulc, work_dir_ucm, aoi, et0, bio_csv)
una_test = una(lulc, work_dir_una, pop, aoi, attr_csv)

## Run InVEST models on clipped samples

In [10]:
# Collect all file names and create fid_list
fid_list = []
for file_name in os.listdir(input_dir):
    if file_name.endswith('.pt'):
        fid = int(file_name.split('.')[0])
        fid_list.append(fid)

In [12]:
from tqdm.notebook import tqdm
import warnings
import logging

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger('pygeoprocessing.geoprocessing').setLevel(logging.ERROR)

os.makedirs(final_output_dir, exist_ok=True)

for fid in tqdm(fid_list, desc="Processing files"):
    try:
        # Load the file
        test_path = f"{input_dir}/{fid}.pt"
        torc = torch.load(test_path, weights_only=True).squeeze(1)
        process_samples(samples, torc, fid, output_dir, thrt_output_dir)

        # Run hq, ucm, and una functions
        hq_test = hq(lulc, work_dir_hq, threats_p, sensitivity_p)
        ucm_test = ucm(lulc, work_dir_ucm, aoi, et0, bio_csv)
        una_test = una(lulc, work_dir_una, pop, aoi, attr_csv)

        # Convert hq_test, ucm_test, and una_test into torch tensors
        hq_tensor = torch.from_numpy(hq_test.values).float()
        ucm_tensor = torch.from_numpy(ucm_test.values).float()
        una_tensor = torch.from_numpy(una_test.values).float()

        # Stack the tensors (3, 512, 512)
        stacked_tensor = torch.stack([hq_tensor, ucm_tensor, una_tensor], dim=0)

        # Save the stacked tensor as <fid>_res.pt in the output directory
        final_output_path = f"{final_output_dir}/{fid}_res.pt"
        torch.save(stacked_tensor, final_output_path)

    except Exception as e:
        # Handle the warning or exception and save the problematic file
        error_path = f"{error_pop}/{fid}_res_e.pt"
        torch.save({'fid': fid, 'error': str(e)}, error_path)
        print(f"Warning/Exception encountered for {fid}: {e}")

Processing files:   0%|          | 0/10 [00:00<?, ?it/s]